In [7]:
import requests
from bs4 import BeautifulSoup

In [237]:
def crawl_data(url):
    response = requests.get(url)
    result = {
            "name": "",
            "address": "",
            "district": "",
            "city": "",
            "main_opening_hour": None,
            "main_closing_hour": None,
            "sub_opening_hour": None,
            "sub_closing_hour": None,
            "category": None,
            "food_categories": [],
            "suitable_time": [],
            "style": [],
            "appropriate": [],
            "average_price_min": 0.0,
            "avarage_price_max": 0.0,
            "average_rating": 0.0,
            "quality_rating": 0.0,
            "location_rating": 0.0,
            "price_rating": 0.0,
            "service_rating": 0.0,
            "space_rating": 0.0,
            "comment_quantity": 0,
            "marvelous_comment": 0,
            "good_comment": 0,
            "ok_comment": 0,
            "awful_comment": 0
        }
    
    if response.status_code==200:
        soup = BeautifulSoup(response.content, 'html.parser')

        result["name"] = soup.find('div', class_='main-info-title').find('h1').text.strip()
        result["address"] = soup.find('div', class_='res-common-add').find_all('span')[1].find('span').text.strip()
        result["district"] = soup.find('div', class_='res-common-add').find_all('span')[4].text.strip()
        result["city"] = soup.find('div', class_='res-common-add').find_all('span')[5].text.strip()

        table_info = soup.find('div', class_='microsite-res-info')
        restaurant_information = table_info.find_all('div', class_='new-detail-info-area')

        for index, value in enumerate(restaurant_information):
            label_text = value.find('div', class_='new-detail-info-label').text.strip()
            
            if "Thời gian hoạt động" in label_text:
                hours = restaurant_information[index].find_all('span')[1].text.strip().split(' | ')
                if(len(hours) == 2):
                    main_hour = hours[0].split(' - ')
                    sub_hour = hours[1].split(' - ')
                    if(len(main_hour) == 2):
                        result["main_opening_hour"] = main_hour[0]
                        result["main_closing_hour"] = main_hour[1]
                    else:
                        result["main_opening_hour"] = "00:00"
                        result["main_closing_hour"] = "00:00" 
                    if(len(sub_hour) == 2):
                        result["sub_opening_hour"] = sub_hour[0]
                        result["sub_closing_hour"] = sub_hour[1]
                    else:
                        result["sub_opening_hour"] = "00:00"
                        result["sub_closing_hour"] = "00:00" 
                else:
                    main_hour = hours[0].split(' - ')
                    if(len(main_hour) == 2):
                        result["main_opening_hour"] = main_hour[0]
                        result["main_closing_hour"] = main_hour[1]
                    else:
                        result["main_opening_hour"] = "00:00"
                        result["main_closing_hour"] = "00:00" 
            elif "Thích hợp" in label_text:
                restaurant_suitable_time = []
                for suitable in restaurant_information[index].find_all('a'):
                    if(suitable.text.strip() != ''):
                        restaurant_suitable_time.append(suitable.text.strip())
                result["suitable_time"] = restaurant_suitable_time
            elif "Thể loại" in label_text:
                result["category"] = restaurant_information[index].find('a').text.strip()
            elif "Phục vụ các món" in label_text:
                restaurant_food_categories = []
                for food_category in restaurant_information[index].find_all('a'):
                    if(food_category.text.strip() != ''):
                        restaurant_food_categories.append(food_category.text.strip())
                result["food_categories"] = restaurant_food_categories
            elif "Phong cách ẩm thực" in label_text:
                restaurant_style = []
                for style in restaurant_information[index].find_all('a'):
                    if(style.text.strip() != ''):
                        restaurant_style.append(style.text.strip())
                result["style"] = restaurant_style
            elif "Phù hợp với" in label_text:
                restaurant_appropriate = []
                for appropriate in restaurant_information[index].find_all('a'):
                    if(appropriate.text.strip() != ''):
                        restaurant_appropriate.append(appropriate.text.strip())
                result["appropriate"] = restaurant_appropriate
        
        rating_information = soup.find('div', class_='microsite-point-group').find_all('div', class_='microsite-top-points') 
        result["quality_rating"] = float(rating_information[0].find('span').text.strip())
        result["location_rating"] = float(rating_information[1].find('span').text.strip())
        result["price_rating"] = float(rating_information[2].find('span').text.strip())
        result["service_rating"] = float(rating_information[3].find('span').text.strip())
        result["space_rating"] = float(rating_information[4].find('span').text.strip())
        result["comment_quantity"] = int(soup.find('div', class_='microsite-top-points-block').find('div', class_='microsite-review-count').text.strip().replace(',', ''))
        result["average_rating"] = float(soup.find('div', class_='microsite-top-points-block').find('div', class_='microsite-point-avg').text.strip())
        
        comment_information = soup.find('div', class_='microsite-box-rating-new').find('div', class_='ratings-boxes').find_all('div', class_='ratings-numbers')
        result["marvelous_comment"] = int(comment_information[0].find('span').find('b').text.strip().replace(',', ''))
        result["good_comment"] = int(comment_information[1].find('span').find('b').text.strip().replace(',', ''))
        result["ok_comment"] = int(comment_information[2].find('span').find('b').text.strip().replace(',', ''))
        result["awful_comment"] = int(comment_information[3].find('span').find('b').text.strip().replace(',', ''))
        
        price_information = soup.find('div', class_='res-common-price').find('div', class_='res-common-minmaxprice').find_all('span')[1].text.strip()
        if price_information != "Đang cập nhật":
            parts = price_information.split(' - ')
            result["average_price_min"] = float(parts[0].replace('.', '').replace('đ', '').strip())
            result["avarage_price_max"] = float(parts[1].replace('.', '').replace('đ', '').strip())
        
    return result


In [238]:
urls = [
    'https://www.foody.vn/ho-chi-minh/mi-dau-hao-hongkong-thuong-hang-%E6%B8%AF%E5%BC%8F%E8%9A%9D%E6%B2%B9%E6%8D%9E%E9%9D%A2-chachaanteng-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/dau-hu-ca-vien-hongkong-%E6%B8%AF%E5%BC%8F%E9%B1%BC%E4%B8%B8%E8%B1%86%E8%85%90-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/kaya-toast-nuong-ca-vien-%E5%92%96%E6%A4%B0%E5%90%90%E5%8F%B8-%E9%B1%BC%E4%B8%B8-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/american-hotdog',
    'https://www.foody.vn/ho-chi-minh/%E8%8C%B6%E9%A4%90%E5%8E%85-cha-chaan-teng-am-thuc-hongkong',
    'https://www.foody.vn/ho-chi-minh/%E6%B8%AF%E5%BC%8F%E7%94%9C%E5%93%81%E5%BA%97-troi-nuoc-hongkong-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/%E6%B8%AF%E5%BC%8F%E7%82%B9%E5%BF%83%E7%81%AB%E9%94%85-dimsum-lau-hai-san-hongkong',
    'https://www.foody.vn/ho-chi-minh/%E5%90%90%E5%8F%B8%E5%B7%A5%E5%9D%8A-%E5%A5%B6%E8%8C%B6%E6%A1%A3-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/che-hongkong-%E6%B8%AF%E5%BC%8F%E7%B3%96%E6%B0%B4-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/xoi-cade-tran-phu-chinh-goc-%E5%92%96%E6%A4%B0%E7%B3%AF%E7%B1%B3%E9%A5%AD',
    'https://www.foody.vn/ho-chi-minh/lau-ca-thac-lac-tuoi-dac-biet-phan-phu-tien.jj93kq',
    'https://www.foody.vn/ho-chi-minh/steak-zone',
    'https://www.foody.vn/ho-chi-minh/truong-thanh-com-ga-xoi-mo-le-thi-hong',
    'https://www.foody.vn/ho-chi-minh/pha-lau-rubi',
    'https://www.foody.vn/ho-chi-minh/banh-mi-t-phan-dang-luu',
    'https://www.foody.vn/ho-chi-minh/sui-cao-thien-thien-ha-ton-quyen',
    'https://www.foody.vn/ho-chi-minh/mi-kho-xa-xiu-gia-phuc',
    'https://www.foody.vn/ho-chi-minh/pizza-hut-thong-nhat',
    'https://www.foody.vn/ho-chi-minh/oc-cam-tan-my',
    'https://www.foody.vn/ho-chi-minh/domino-s-pizza-nguyen-duy-trinh',
    'https://www.foody.vn/ho-chi-minh/che-thai-ut-lyly',
    'https://www.foody.vn/ho-chi-minh/banh-canh-cua-260-tan-son-nhi',
    'https://www.foody.vn/ho-chi-minh/banh-trang-toi-019',
    'https://www.foody.vn/ho-chi-minh/tra-sua-maycha-dong-den',
    'https://www.foody.vn/ho-chi-minh/hoa-co-com-van-phong-nguyen-van-sang',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-ba-mien',
    'https://www.foody.vn/ho-chi-minh/bun-quay-kien-xay-dien-bien-phu',
    'https://www.foody.vn/ho-chi-minh/gio-cha-minh-chau',
    'https://www.foody.vn/ho-chi-minh/3a-korean-food-ga-sot-pho-mai-mi-cay-han-quoc-tokbokki-nguyen-van-luong',
    'https://www.foody.vn/ho-chi-minh/com-tam-tai-476-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/mr-nghiem-saigon-chao-ech-chao-cua-singapore-nguyen-huu-tho',
    'https://www.foody.vn/ho-chi-minh/anh-cuong-bakery-banh-kem-banh-mi-nguyen-van-lac',
    'https://www.foody.vn/ho-chi-minh/tau-hu-coc-hoa-su',
    'https://www.foody.vn/ho-chi-minh/tra-va-sua',
    'https://www.foody.vn/ho-chi-minh/co-thuy-mi-xao-gion',
    'https://www.foody.vn/ho-chi-minh/chao-trang-hot-vit-muoi-tran-khac-chan',
    'https://www.foody.vn/ho-chi-minh/sui-cao-dai-nuong',
    'https://www.foody.vn/ho-chi-minh/hien-banh-canh-cua',
    'https://www.foody.vn/ho-chi-minh/tu-thuy-mi-hen',
    'https://www.foody.vn/ho-chi-minh/bun-dau-mac-van-khoa-ten-lua',
    'https://www.foody.vn/ho-chi-minh/com-nieu-phuong-nam-com-van-phong',
    'https://www.foody.vn/ho-chi-minh/lotteria-ly-thai-to',
    'https://www.foody.vn/ho-chi-minh/tra-sua-kem-tuyet-hong-phuc-togo',
    'https://www.foody.vn/ho-chi-minh/kfc-xo-viet-nghe-tinh',
    'https://www.foody.vn/ho-chi-minh/phuc-long-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/baoz-dimsum-resraurant',
    'https://www.foody.vn/ho-chi-minh/chi-bo-quan-bun-bo',
    'https://www.foody.vn/ho-chi-minh/starbucks-coffee-lotte-nsg',
    'https://www.foody.vn/ho-chi-minh/com-chay-phuoc-hoa',
    'https://www.foody.vn/ho-chi-minh/ca-nuong-mien-trung',
    'https://www.foody.vn/ho-chi-minh/me-vui-29-tran-quoc-toan',
    'https://www.foody.vn/ho-chi-minh/banh-mi-to-bo-kho-xiu-mai-lagu-ga-duong-d5',
    'https://www.foody.vn/ho-chi-minh/com-ga-hai-nam-hau-giang',
    'https://www.foody.vn/ho-chi-minh/chi-ut-vu-heo-nuong-minh-phung',
    'https://www.foody.vn/ho-chi-minh/phuc-long-ly-thuong-kiet',
    'https://www.foody.vn/ho-chi-minh/seva-nha-hang-chay',
    'https://www.foody.vn/ho-chi-minh/tra-sua-tocotoco-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/kim-huong-bun-bo-hue-nguyen-xi',
    'https://www.foody.vn/ho-chi-minh/com-tam-06',
    'https://www.foody.vn/ho-chi-minh/co-ba-che-com-tam-long-xuyen-nguyen-xi',
    'https://www.foody.vn/ho-chi-minh/che-via-he-vo-van-tan',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-335',
    'https://www.foody.vn/ho-chi-minh/lau-ca-keo-soc-trang',
    'https://www.foody.vn/ho-chi-minh/ga-ta-que-nha',
    'https://www.foody.vn/ho-chi-minh/milo-dam-chung-cu-42',
    'https://www.foody.vn/ho-chi-minh/tiramisu-kieu-italy-nguyen-ban-thu-cong-breakfast-shop-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/che-nguoi-hoa-an-diem',
    'https://www.foody.vn/ho-chi-minh/quan-com-thin-ky',
    'https://www.foody.vn/ho-chi-minh/hong-tra-ngo-gia-cu-xa-phu-lam-b',
    'https://www.foody.vn/ho-chi-minh/tu-anh-com-ga-xoi-mo',
    'https://www.foody.vn/ho-chi-minh/bun-rieu-banh-da-cua-cha-ngo-thi-thu-minh',
    'https://www.foody.vn/ho-chi-minh/bun-oc-rieu-cua-thanh-hai-ky-dong',
    'https://www.foody.vn/ho-chi-minh/com-tam-449-an-duong-vuong',
    'https://www.foody.vn/ho-chi-minh/ha-cao-ngoc-lan',
    'https://www.foody.vn/ho-chi-minh/hai-ky-mi-gia-nguyen-trai',
    'https://www.foody.vn/ho-chi-minh/din-chon-mi-gia',
    'https://www.foody.vn/ho-chi-minh/lau-bo-co-thao-ho-thi-ky',
    'https://www.foody.vn/ho-chi-minh/vinh-an-chay-com-suon-bi-cha-chay-bun-thit-nuong-cha-gio-chay-au-co',
    'https://www.foody.vn/ho-chi-minh/domino-s-pizza-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/tuktuk-thai-bistro-ly-tu-trong',
    'https://www.foody.vn/ho-chi-minh/tri-phuong-lau-bo',
    'https://www.foody.vn/ho-chi-minh/mutsumian-luon-nuong-than',
    'https://www.foody.vn/ho-chi-minh/a-ma-kitchen-mon-hongkong',
    'https://www.foody.vn/ho-chi-minh/tau-hu-an-chi',
    'https://www.foody.vn/ho-chi-minh/kim-huong-bun-bo-hue-hoang-hoa-tham',
    'https://www.foody.vn/ho-chi-minh/com-tam-co-ba-sai-gon-le-quang-dinh',
    'https://www.foody.vn/ho-chi-minh/xoi-xoi-xoi-xien-nuong',
    'https://www.foody.vn/ho-chi-minh/kitachi-mi-cay-han-quoc-7-cap-do-155-van-kiep',
    'https://www.foody.vn/ho-chi-minh/quan-thao-hien-dac-san-mien-trung',
    'https://www.foody.vn/ho-chi-minh/sakura-udon-japanese-fast-food',
    'https://www.foody.vn/ho-chi-minh/hai-cay-dua',
    'https://www.foody.vn/ho-chi-minh/banh-mi-hoang-phat',
    'https://www.foody.vn/ho-chi-minh/tien-hai-quan-bun-dau-mam-tom-ha-noi',
    'https://www.foody.vn/ho-chi-minh/tiem-nha-ngan-tra-trai-cay-an-vat',
    'https://www.foody.vn/ho-chi-minh/com-ga-singapore-quan-3-214c-ly-chinh-thang',
    'https://www.foody.vn/ho-chi-minh/tuyet-nhung-com-ga-phu-yen',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-tra-ca-phe-banh-402-tran-hung-dao-b',
    'https://www.foody.vn/ho-chi-minh/busan-korean-food-mon-han-quoc-dong-nai',
    'https://www.foody.vn/ho-chi-minh/com-tam-phuc-loc-tho-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/hu-tieu-co-ba-1588-huynh-tan-phat'
]

results = []

for url in urls:
    data = crawl_data(url)
    if data:
        results.append(data)


In [239]:
import json
with open("restaurants.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)
print("✅ Đã lưu dữ liệu vào restaurants.json")

✅ Đã lưu dữ liệu vào restaurants.json
